In [55]:
#libraries
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [85]:
#for generating Train directory structure
new_path = Path(str('Data/Train/'))
new_path.mkdir(parents = True, exist_ok = True)
new_path.joinpath('0').mkdir(parents = True, exist_ok = True)
new_path.joinpath('1').mkdir(parents = True, exist_ok = True)
new_path.joinpath('2').mkdir(parents = True, exist_ok = True)
new_path.joinpath('3').mkdir(parents = True, exist_ok = True)
new_path.joinpath('4').mkdir(parents = True, exist_ok = True)

In [86]:
#for generating Train directory structure
new_path = Path(str('Data/Valid/'))
new_path.mkdir(parents = True, exist_ok = True)
new_path.joinpath('0').mkdir(parents = True, exist_ok = True)
new_path.joinpath('1').mkdir(parents = True, exist_ok = True)
new_path.joinpath('2').mkdir(parents = True, exist_ok = True)
new_path.joinpath('3').mkdir(parents = True, exist_ok = True)
new_path.joinpath('4').mkdir(parents = True, exist_ok = True)

In [87]:
#for generating Test directory structure
new_path = Path(str('Data/Test/'))
new_path.mkdir(parents = True, exist_ok = True)
new_path.joinpath('0').mkdir(parents = True, exist_ok = True)
new_path.joinpath('1').mkdir(parents = True, exist_ok = True)
new_path.joinpath('2').mkdir(parents = True, exist_ok = True)
new_path.joinpath('3').mkdir(parents = True, exist_ok = True)
new_path.joinpath('4').mkdir(parents = True, exist_ok = True)

In [ ]:
#read the data file
df = pd.read_csv("mitbih_train.csv",header=None)
#save the png files in their appropriate folders
for i in range(5):
    #since the last col represents label
    x=df.loc[df[187]==i].values.tolist()
    count=0
    for j in range(int(len(x)*0.8)):
        fig = plt.figure()
        #remove all the border lines
        figure = plt.gca()
        x_axis = figure. axes. get_xaxis()
        x_axis. set_visible(False)
        y_axis = figure. axes. get_yaxis()
        y_axis. set_visible(False)
        plt.plot(x[j][0:-1])
        plt.savefig('Data/Train/'+str(i)+'/'+str(count)+'.png',bbox_inches='tight',pad_inches = 0)
        count=count+1
    count=0
    for k in range(int(len(x)*0.8),len(x)):
        fig = plt.figure()
        #remove all the border lines
        figure = plt.gca()
        x_axis = figure. axes. get_xaxis()
        x_axis. set_visible(False)
        y_axis = figure. axes. get_yaxis()
        y_axis. set_visible(False)
        plt.plot(x[j][0:-1])
        plt.savefig('Data/Valid/'+str(i)+'/'+str(count)+'.png',bbox_inches='tight',pad_inches = 0)
        count=count+1

C:\Users\KrishnaS\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


In [ ]:
#read the data file
df = pd.read_csv("mitbih_test.csv",header=None)
#save the png files in their appropriate folders
for i in range(5):
    #since the last col represents label
    x=df.loc[df[187]==i].values.tolist()
    count=0
    for j in range(len(x)):
        fig = plt.figure()
        #remove all the border lines
        figure = plt.gca()
        x_axis = figure. axes. get_xaxis()
        x_axis. set_visible(False)
        y_axis = figure. axes. get_yaxis()
        y_axis. set_visible(False)
        plt.plot(x[j][0:-1])
        plt.savefig('Data/Test/'+str(i)+'/'+str(count)+'.png',bbox_inches='tight',pad_inches = 0)
        count=count+1

In [ ]:
#for classification
# Imports here
import torch
import PIL
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
from collections import OrderedDict
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
data_dir = 'Data'
train_dir = '/Train'
valid_dir = data_dir + '/Valid'
test_dir = data_dir + '/Test'

In [ ]:
# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])


valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=75, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=75)
testloader = torch.utils.data.DataLoader(test_data, batch_size=75)

In [ ]:
model = models.vgg16(pretrained=True)
model.name = "vgg16"

In [ ]:
for param in model.parameters():
    param.requires_grad = False
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096, bias=True)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.5)),
                          ('fc2', nn.Linear(4096, 5, bias=True)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device);

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
epochs = 200
print_every = 10
steps = 0

In [ ]:
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels) in enumerate(testloader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy
print("Training process initializing .....\n")

for e in range(epochs):
    running_loss = 0
    model.train() # Technically not necessary, setting this for good measure
    
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()

            with torch.no_grad():
                valid_loss, accuracy = validation(model, validloader, criterion)
            
            print("Epoch: {}/{} | ".format(e+1, epochs),
                  "Training Loss: {:.4f} | ".format(running_loss/print_every),
                  "Validation Loss: {:.4f} | ".format(valid_loss/len(testloader)),
                  "Validation Accuracy: {:.4f}".format(accuracy/len(testloader)))
            
            running_loss = 0
            model.train()

print("\nTraining process is now complete!!")

In [ ]:
#for testing
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct / total))

In [ ]:
#to save the check point
model.class_to_idx = train_data.class_to_idx
checkpoint = {'architecture': model.name,
             'classifier': model.classifier,
             'class_to_idx': model.class_to_idx,
             'state_dict': model.state_dict()}

torch.save(checkpoint, 'my_checkpoint.pth')